In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [13]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = dict(sorted({k: i for i, k in enumerate(world_set)}.items(), key = lambda x : x[1]))

In [14]:
print(vocab)

{'b': 0, 'a': 1, ',': 2, 'c': 3, ' ': 4, 'y': 5, 'm': 6, 'p': 7, 'h': 8, '.': 9, 'r': 10, 'e': 11, 'i': 12, 'o': 13, 'k': 14, 'w': 15, 'u': 16, "'": 17, 'g': 18, 'f': 19, 'B': 20, 'l': 21, 't': 22, 's': 23, 'd': 24, 'n': 25}


In [15]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [16]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [18]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : sequence_length + i]
  y_str = sentence[i+1 : sequence_length+i+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [19]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[20, 10, 12, 3, 14, 4, 15, 1, 21, 21]
[10, 12, 3, 14, 4, 15, 1, 21, 21, 23]


In [30]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) ## 연산을 위한 float 형태
Y = torch.LongTensor(y_data) ## label이므로 정수형

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [31]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [32]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [33]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([10, 12,  3, 14,  4, 15,  1, 21, 21, 23])


In [34]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)
  
  def forward(self, x):
    hidden_0 = torch.zeros(2, len(x), hidden_size, requires_grad = True) # 첫번째 층 0으로 초기화
    x,_ = self.rnn(x, hidden_0) #batch_first로 나와서 굳이 transpose할 필요 없다.
    x = self.fc(x)
    return x

In [35]:
net = Net(vocab_size, hidden_size, 2)

In [36]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [37]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [44]:
outputs.view(-1,26).shape

torch.Size([1880, 26])

In [45]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, 26), Y.view(-1))
    #X shape : batchesxfeatures, output classes / Y : batches
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2) # 26개의 글자중 제일 높은 확률을 가지는 것
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

hlhlhahhhhhhhlhahhhyhahhlhahhhahhyatlahyhyhhhhhhyahhhhhyahhhtyhhhhyyhhlhhhhhhhhhlhahhhhhhyhahhhacaehhhhhayyhhhyhhalhhyahaahhhhhhayhhhhyhhhhyyhhlhhhhhhhhhlhahhhyhahhhhhhhhhhhhhhhyaacyhhhyhhyhhhhhyyh
h                                                                                               aa                                                                                  a    a           
t                                                                                                                                                                                                    
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [46]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinkss"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

####1. 기본 베이스라인

In [47]:
sentence = ("이제 난 시간 여행을 하지 않는다, 하루를 위해서라도"
            "그저 내가 이날을 위해 시간 여행을 한 것처럼 즐겁게 매일 지내려고 노력할 뿐이다."
            "우리가 할 수 있는 최선은 이 멋진 여행을 즐기는 것 뿐이다.")

In [48]:
world_set = list(set(sentence))

In [49]:
vocab = vocab = dict(sorted({k: i for i, k in enumerate(world_set)}.items(), key = lambda x : x[1]))

In [50]:
print(vocab)

{'멋': 0, '리': 1, '진': 2, ',': 3, '를': 4, '라': 5, '것': 6, '가': 7, '노': 8, '력': 9, '위': 10, '행': 11, '선': 12, '다': 13, '뿐': 14, ' ': 15, '일': 16, '여': 17, '서': 18, '.': 19, '간': 20, '최': 21, '을': 22, '은': 23, '도': 24, '그': 25, '루': 26, '할': 27, '이': 28, '해': 29, '내': 30, '제': 31, '게': 32, '시': 33, '수': 34, '우': 35, '지': 36, '럼': 37, '않': 38, '처': 39, '즐': 40, '하': 41, '난': 42, '고': 43, '저': 44, '한': 45, '려': 46, '날': 47, '매': 48, '있': 49, '는': 50, '겁': 51, '기': 52}


In [51]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 53


In [53]:
#하이퍼 파라미터
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15 # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [54]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : sequence_length + i]
  y_str = sentence[i+1 : sequence_length+i+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 이제 난 시간 여행을 하지  -> 제 난 시간 여행을 하지 않
1 제 난 시간 여행을 하지 않 ->  난 시간 여행을 하지 않는
2  난 시간 여행을 하지 않는 -> 난 시간 여행을 하지 않는다
3 난 시간 여행을 하지 않는다 ->  시간 여행을 하지 않는다,
4  시간 여행을 하지 않는다, -> 시간 여행을 하지 않는다, 
5 시간 여행을 하지 않는다,  -> 간 여행을 하지 않는다, 하
6 간 여행을 하지 않는다, 하 ->  여행을 하지 않는다, 하루
7  여행을 하지 않는다, 하루 -> 여행을 하지 않는다, 하루를
8 여행을 하지 않는다, 하루를 -> 행을 하지 않는다, 하루를 
9 행을 하지 않는다, 하루를  -> 을 하지 않는다, 하루를 위
10 을 하지 않는다, 하루를 위 ->  하지 않는다, 하루를 위해
11  하지 않는다, 하루를 위해 -> 하지 않는다, 하루를 위해서
12 하지 않는다, 하루를 위해서 -> 지 않는다, 하루를 위해서라
13 지 않는다, 하루를 위해서라 ->  않는다, 하루를 위해서라도
14  않는다, 하루를 위해서라도 -> 않는다, 하루를 위해서라도그
15 않는다, 하루를 위해서라도그 -> 는다, 하루를 위해서라도그저
16 는다, 하루를 위해서라도그저 -> 다, 하루를 위해서라도그저 
17 다, 하루를 위해서라도그저  -> , 하루를 위해서라도그저 내
18 , 하루를 위해서라도그저 내 ->  하루를 위해서라도그저 내가
19  하루를 위해서라도그저 내가 -> 하루를 위해서라도그저 내가 
20 하루를 위해서라도그저 내가  -> 루를 위해서라도그저 내가 이
21 루를 위해서라도그저 내가 이 -> 를 위해서라도그저 내가 이날
22 를 위해서라도그저 내가 이날 ->  위해서라도그저 내가 이날을
23  위해서라도그저 내가 이날을 -> 위해서라도그저 내가 이날을 
24 위해서라도그저 내가 이날을  -> 해서라도그저 내가 이날을 위
25 해서라도그저 내가 이날을 위 -> 서라도그저 내가 이날을 위해
26 서라도그저 내가 이날을 위해 -> 

In [55]:
print(x_data[0])
print(y_data[0])

[28, 31, 15, 42, 15, 33, 20, 15, 17, 11, 22, 15, 41, 36, 15]
[31, 15, 42, 15, 33, 20, 15, 17, 11, 22, 15, 41, 36, 15, 38]


In [56]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot) ## 연산을 위한 float 형태
Y = torch.LongTensor(y_data) ## label이므로 정수형

In [57]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([94, 15, 53])
레이블의 크기 : torch.Size([94, 15])


In [58]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)
  
  def forward(self, x):
    hidden_0 = torch.zeros(2, len(x), hidden_size, requires_grad = True) # 첫번째 층 0으로 초기화
    x,_ = self.rnn(x, hidden_0) #batch_first로 나와서 굳이 transpose할 필요 없다.
    x = self.fc(x)
    return x

In [59]:
net = Net(vocab_size, hidden_size, 2)

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [60]:
outputs = net(X)
print(outputs.shape)

torch.Size([94, 15, 53])


In [62]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1,vocab_size), Y.view(-1))
    #X shape : batchesxfeatures, output classes / Y : batches
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2) # 26개의 글자중 제일 높은 확률을 가지는 것
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

일일일일일 것일 일일일 일일처 처처일루일일겁 일일일일일처  일일 일일일 일일 것일 일일일 일일일일일 일처 일일일 일일일 처난일일 일일처일일처일 일 일겁일일처일일겁 일 일일일일일일 일일일일일 일일
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
                                                                                                            
         을                                    을 하   을               을                           을           
        행을 여   해  하  여                하      행을 여   을     행         을           여     행  하  행  행을 여   하   을 
        행을 여      내

In [63]:
predict_str

'  난 시간 여행을 하지 않는다, 하루를 위해서라도그저 내가 이날을 위해 시간 여행을 한 것처럼 즐겁게 매일 지내려고 노력할 뿐이다.우리가 할 수 있는 최선은 이 멋진 여행을 즐기는 것 뿐이다.'

정확한 문장 생성이 되었다.

In [64]:
print(sentence)

이제 난 시간 여행을 하지 않는다, 하루를 위해서라도그저 내가 이날을 위해 시간 여행을 한 것처럼 즐겁게 매일 지내려고 노력할 뿐이다.우리가 할 수 있는 최선은 이 멋진 여행을 즐기는 것 뿐이다.
